## Download The Dataset

In [ ]:
## step number one

from IPython.display import clear_output
!pip install --upgrade kaggle
clear_output()
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 68 bytes


In [ ]:
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d firefliesqn/zalo2022-liveness

 99% 2.42G/2.44G [00:25<00:00, 121MB/s]
100% 2.44G/2.44G [00:25<00:00, 103MB/s]


In [ ]:
!unzip -xq '/content/zalo2022-liveness.zip'

In [ ]:
!rm -rf '/content/zalo2022-liveness.zip'

## Load

In [ ]:
!pip install av

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 38.9 MB/s eta 0:00:00


In [ ]:
!pip install git+https://github.com/hassony2/torch_videovision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/hassony2/torch_videovision to /tmp/pip-req-build-h1a17ps6
  Running command git clone --filter=blob:none --quiet https://github.com/hassony2/torch_videovision /tmp/pip-req-build-h1a17ps6
  Resolved https://github.com/hassony2/torch_videovision to commit 785a810d1df2ccab6901fb98b38e358433e7b17a
  Preparing metadata (setup.py) ... done
  Created wheel for torchvideotransforms: filename=torchvideotransforms-0.1.2-py3-none-any.whl size=23769 sha256=6dbba8c988c9af2747dc335749e572dbb29a9793e04b4e6494fe64077da6840a
  Stored in directory: /tmp/pip-ephem-wheel-cache-6f_uuoo7/wheels/a3/1f/41/d882524ea6c13f4fed0915cf5012d34bf18b753a83b9a651cd
Successfully built torchvideotransforms


In [ ]:
import torchvision
from torchvision.datasets.video_utils import read_video
import torch
from torchvision import transforms
import pandas as pd
import os
from glob import glob
from shutil import copy
from tqdm.notebook import tqdm
import torch
from torchvision.datasets.video_utils import VideoClips
import cv2
from torchvideotransforms import video_transforms, volume_transforms
import torch.nn as nn

In [ ]:
video = read_video('/content/liveness_test/public/videos/0.mp4')

/usr/local/lib/python3.8/dist-packages/torchvision/io/video.py:162: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


In [ ]:
video[0].shape

torch.Size([152, 1280, 720, 3])

## Change Dataset format

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 16
IMAGE_SIZE = 50
NUM_EPOCHS = 100
SEED = 101

In [ ]:
train_videos_real = '/content/videos/train/real'
os.makedirs(train_videos_real, exist_ok = True)
train_videos_spoof = '/content/videos/train/spoof'
os.makedirs(train_videos_spoof, exist_ok = True)


test_videos_real = '/content/videos/test/real'
os.makedirs(test_videos_real, exist_ok = True)
test_videos_spoof = '/content/videos/test/spoof'
os.makedirs(test_videos_spoof, exist_ok = True)

In [ ]:
labels = pd.read_csv('/content/liveness_train/train/label.csv')
labels.head()

,fname,liveness_score
0,1.mp4,0
1,2.mp4,1
2,3.mp4,1
3,5.mp4,0
4,7.mp4,1


In [ ]:
len(labels)

1168

In [ ]:
labels[labels['liveness_score'] == 1]

,fname,liveness_score
1,2.mp4,1
2,3.mp4,1
4,7.mp4,1
5,8.mp4,1
7,11.mp4,1
...,...,...
1159,1874.mp4,1
1160,1876.mp4,1
1161,1877.mp4,1
1162,1879.mp4,1


In [ ]:
videos_path = os.listdir('/content/liveness_train/train/videos')
len(videos_path)

1168

In [ ]:
real_videos = list(labels[labels['liveness_score'] == 1]['fname'])
train_real_videos = real_videos[:int(0.7*len(real_videos))]
val_real_videos   = real_videos[int(0.7*len(real_videos)) :]


spoof_videos = list(labels[labels['liveness_score'] == 0]['fname'])
train_spoof_videos = spoof_videos[:int(0.7*len(spoof_videos))]
val_spoof_videos   = spoof_videos[int(0.7*len(spoof_videos)):]

In [ ]:
for videos_name in tqdm(train_real_videos):
  copy(os.path.join('/content/liveness_train/train/videos', videos_name), '/content/videos/train/real')

for videos_name in tqdm(train_spoof_videos):
  copy(os.path.join('/content/liveness_train/train/videos', videos_name), '/content/videos/train/spoof')

  0%|          | 0/418 [00:00<?, ?it/s]

  0%|          | 0/399 [00:00<?, ?it/s]

In [ ]:
for videos_name in tqdm(val_real_videos):
  copy(os.path.join('/content/liveness_train/train/videos', videos_name), '/content/videos/test/real')

for videos_name in tqdm(val_spoof_videos):
  copy(os.path.join('/content/liveness_train/train/videos', videos_name), '/content/videos/test/spoof')

  0%|          | 0/180 [00:00<?, ?it/s]

  0%|          | 0/171 [00:00<?, ?it/s]

## Custiom Dataset

In [ ]:
class  SpoofingDataset(torch.utils.data.Dataset):
  def __init__(self, dataset_path, phase, FPS = 4):
    self.FPS = FPS
    self.dataset    = torchvision.datasets.DatasetFolder(os.path.join(dataset_path, phase), loader=read_video, extensions = 'mp4')
    self.video_list = [x[0] for x in self.dataset.samples]
    self.labels     = [x[1] for x in self.dataset.samples]
    
    self.video_clips = VideoClips(video_paths= self.video_list, clip_length_in_frames= self.FPS)
    self.video_transform_list = [
          video_transforms.Resize((IMAGE_SIZE,IMAGE_SIZE)),
          volume_transforms.ClipToTensor()
      ]  
    self.transforms = video_transforms.Compose(self.video_transform_list)
    
    
  def __getitem__(self,item):
    video , audio, fps, video_idx = self.video_clips.get_clip(item)
    video = video.cpu().detach().numpy()
    
    label = self.labels[video_idx]

    # print('item:' ,item)
    # print('video_idx: ',video_idx)
    # print('dir: ',self.video_list[video_idx])
    # print('label: ', label)

    if self.transforms is not None:
      video = self.transforms(video)
    return video , label


    
  def __len__(self):
    return len(self.video_list)


In [ ]:
spoofdetection_train = SpoofingDataset(dataset_path='/content/videos', phase = 'train', FPS = 4)

  0%|          | 0/52 [00:00<?, ?it/s]

In [ ]:
spoofdetection_test = SpoofingDataset(dataset_path='/content/videos', phase = 'test', FPS = 4)

  0%|          | 0/22 [00:00<?, ?it/s]

In [ ]:
class Costum_Net(nn.Module):
  def __init__(self):
    super(Costum_Net,self).__init__()
    self.base = torchvision.models.video.r3d_18(True)
    self.base.fc = nn.Linear(in_features=512, out_features= 1)
    self.activation = nn.Sigmoid()

  def forward(self, input):
    x = self.base(input)
    x = self.activation(x)
    return(x)

In [ ]:
model = Costum_Net().to(device)
model

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Costum_Net(
  (base): VideoResNet(
    (stem): BasicStem(
      (0): Conv3d(3, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2), padding=(1, 3, 3), bias=False)
      (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Sequential(
          (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
          (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (conv2): Sequential(
          (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
          (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (relu): ReLU(inplace=True)
      )
      (1): BasicBlock(
        (conv1): Sequential(
          (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3

In [ ]:
from torch.utils.data import Dataset, DataLoader
test_data_loader  = DataLoader(spoofdetection_test,  batch_size=BATCH_SIZE, drop_last = True, shuffle= True)
train_data_loader = DataLoader(spoofdetection_train, batch_size=BATCH_SIZE, drop_last = True, shuffle= True)

In [ ]:
X, Y = next(iter(test_data_loader))
print('X shape:', X.shape, X.dtype)
print('Y shape:', Y.shape, Y.dtype)
X, Y = X.to(device), Y.to(device)

X shape: torch.Size([16, 3, 4, 50, 50]) torch.float32
Y shape: torch.Size([16]) torch.int64


In [ ]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.00001)

In [ ]:
output = model(X)
output

tensor([[0.9695],
        [0.1545]], grad_fn=<AddmmBackward0>)

In [ ]:
Y

tensor([0, 0])

In [ ]:
threshold = torch.tensor([0.5]).to(device)
results = (output>threshold).float()*1
results

tensor([[1.],
        [1.]])

In [ ]:
criterion(output, Y.view(-1,1).float())

tensor(1.8285, grad_fn=<BinaryCrossEntropyBackward0>)

In [ ]:
dataloaders ={'train':train_data_loader,
              'val':test_data_loader}

In [ ]:
from tqdm.notebook import tqdm
import time
import copy
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25 ):
    since = time.time()
    loss = 0.0
    correct = 0.0
    iterr_acc = 0.0
    val_acc_history = []
    train_acc_history = []
    threshold = torch.tensor([0.5]).to(device)
    val_loss_history = []
    train_loss_history = []

    # best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        epoch_loss = 0.0
        epoch_acc = 0.0
        
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            # Iterate over data.
            t = tqdm(dataloaders[phase])
            for batch_idx, (inputs, labels) in enumerate(t):

                t.set_description(f"best acc : {best_acc:.2f}, Acc : {iterr_acc:.2f}, loss : {loss:.2f}")
                t.refresh()

                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # print()
                    outputs = model(inputs)
                    # print('output:' ,outputs)

                    results = (outputs>threshold).float()*1
                    # print('results:' ,results)
                    
                    labels = labels.view(-1,1)
                    # print('labels:' ,labels)
                    
                    correct += (results == labels).sum().item()
                    iterr_acc = correct / BATCH_SIZE
                    epoch_acc = iterr_acc + epoch_acc

                    loss = criterion(outputs, labels.view(-1,1).float())
                    epoch_loss = epoch_loss + loss

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        train_acc_history.append(iterr_acc)
                        train_loss_history.append(loss)
                    if phase == 'val' :
                        val_loss_history.append(loss)

                    correct = 0.0
                # statistics
                
            epoch_loss = epoch_loss / (len( dataloaders[phase].dataset ) // BATCH_SIZE)
            epoch_acc = epoch_acc / (len( dataloaders[phase].dataset ) // BATCH_SIZE)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history,val_loss_history, train_loss_history, train_acc_history

In [ ]:
model, val_acc, val_loss, train_loss, train_acc = train_model(model, dataloaders, criterion, optimizer, num_epochs=100)

Epoch 0/99
----------


  0%|          | 0/51 [00:00<?, ?it/s]

train Loss: 0.6819 Acc: 0.5931


  0%|          | 0/21 [00:00<?, ?it/s]

val Loss: 0.8217 Acc: 0.4211

Epoch 1/99
----------


  0%|          | 0/51 [00:00<?, ?it/s]

train Loss: 0.4358 Acc: 0.9816


  0%|          | 0/21 [00:00<?, ?it/s]

KeyboardInterrupt: ignored